In [20]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sys
import bz2
import pickle
import ts_cluster
from multiprocessing.pool import ThreadPool

%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 8]
np.random.seed(2341)
pool = ThreadPool(processes=6)

In [17]:
def read_bz2_ret_pickle(fname=None):
  if fname is None or fname == "":
    raise ValueError("file name does not contain a value!")
  
  with bz2.BZ2File(fname, 'rb') as f:
    f_data = pickle.load(f)
  return f_data

def write_bz2_pickle(data,fname=None):
    if fname is None or fname == "":
      raise ValueError("file name does not contain a value!")
    
    with bz2.BZ2File(fname, 'wb') as f:
      pickle.dump(data,f,pickle.HIGHEST_PROTOCOL)

def read_all_runs_log(fnames):
  runs = []
  for fname in fnames:
    runs.append(read_bz2_ret_pickle(fname))
  return runs

def cluster_param_series(param_series,num_clusters = 10, 
                         clustering_iter = 10, locality_window = 10):
  km_lb = ts_cluster.ts_cluster(num_clusters)
  km_lb.k_means_clust(param_series.tolist(),
                      clustering_iter, locality_window, progress=False)
  return km_lb

def cluster_run(weights,epochs=30,num_clusters = 10, 
                         clustering_iter = 10, locality_window = 10):
  kmeans_l = []
  n_times = len(weights)//epochs
  for i in range(epochs):
    print ('clustering for epoch {}'.format(i))
    curr_weight = weights[i*n_times:min((i+1)*n_times,len(weights))]
    curr_weight = np.array(curr_weight).transpose()
    kmeans_l.append(cluster_param_series(curr_weight,num_clusters,
                                         clustering_iter,locality_window))
  
  return kmeans_l

In [3]:
run_0 = read_bz2_ret_pickle('./madeup_mnist/30_experiments_clean/run_0.pickle.bz2')

In [21]:
print(type(run_0))
print(run_0.keys())
print(type(run_0['weights']))
print(len(run_0['weights']))
print(type(run_0['weights'][0]))
print(run_0['weights'][0].shape)

epochs = 30
n_times = len(run_0['weights'])//epochs
print (n_times)
www = run_0['weights'][0:n_times]
print(len(www))
www = np.array(www).transpose()
print(www.shape)

k_10_itr_25_async = pool.apply_async(cluster_run,(run_0['weights'],30,10,25,10))
k_15_itr_25_async = pool.apply_async(cluster_run,(run_0['weights'],30,15,25,10))
k_10_itr_50_async = pool.apply_async(cluster_run,(run_0['weights'],30,10,50,10))
k_15_itr_50_async = pool.apply_async(cluster_run,(run_0['weights'],30,15,50,10))

k_10_itr_25 = k_10_itr_25_async.get()
k_15_itr_25 = k_15_itr_25_async.get()
k_10_itr_50 = k_10_itr_50_async.get()
k_15_itr_50 = k_15_itr_50_async.get()


<class 'dict'>
dict_keys(['weights', 'poisoned_batch_numbers', 'bacth_losses', 'batch_accuracies', 'test_losses', 'test_accuracies'])
<class 'list'>
14040
<class 'numpy.ndarray'>
(1290,)
468
468
(1290, 468)
clustering for epoch 0clustering for epoch 0
clustering for epoch 0
clustering for epoch 0

clustering for epoch 1
clustering for epoch 1
clustering for epoch 2
clustering for epoch 1
clustering for epoch 1
clustering for epoch 3
clustering for epoch 2
clustering for epoch 2
clustering for epoch 4


KeyboardInterrupt: 

clustering for epoch 3
clustering for epoch 2
